## Don't Run All Make sure to convert to live later

In [1]:
from datetime import datetime
from fund.fund import Fund
from pricer.pricer import Pricer as pricer_list
from ranker.ranker import Ranker as ranker_list
from classifier.classifier import Classifier as classifier_list
from portfolio.aportfolio import APortfolio
from parameters.parameters import Parameters 
import pandas as pd
from returns.products import Products
from processor.processor import Processor as p
from alpaca_api.alpaca_api import AlpacaApi
from time import sleep

In [2]:
alp = AlpacaApi()

In [3]:
portfolio = APortfolio(pricer_list.WEEKLY_STOCK_SPECULATION
                          ,classifier_list.WEEKLY_STOCK_SPECULATION_CLASSIFIER
                          ,ranker_list.WEEKLY_STOCK_ROLLING_RANKER)
portfolios = [portfolio]

In [4]:
backtest_start_date = datetime(2023,1,1)
backtest_end_date  = datetime.now()
current_start_date = datetime.now()

In [5]:
fund = Fund(portfolios,backtest_start_date,backtest_end_date,current_start_date)

In [6]:
fund.initialize_portfolios()
fund.initialize_backtesters()

In [31]:
fund.market.connect()
prices = fund.market.retrieve("stocks")
sp500 = fund.market.retrieve("sp500")
sp500 = sp500.rename(columns={"Symbol":"ticker"})
fund.market.disconnect()

In [19]:
prices = p.column_date_processing(prices)

In [21]:
merged_cols = ['year', 'week', 'ticker', 'price_prediction', 'rank_prediction',
       'classification_prediction', 'quarter', 'risk_return_2', 'risk_return_3',
       'risk_return_4', 'day', 'weekly_risk_return', 'bench_weekly_return',
       'weekly_variance', 'market_weekly_cov', 'weekly_beta']

In [24]:
test_portfolio = fund.portfolios[0]
risk_returns = test_portfolio.create_risk_returns()
sim = test_portfolio.create_current_simulation()
merged = test_portfolio.merge_sim_returns(sim,risk_returns)
sim = prices[["date","year","week","ticker","adjclose"]].merge(merged[merged_cols],on=["year","week","ticker"],how="left").dropna()

In [25]:
sim

,date,year,week,ticker,adjclose,price_prediction,rank_prediction,classification_prediction,quarter,risk_return_2,risk_return_3,risk_return_4,day,weekly_risk_return,bench_weekly_return,weekly_variance,market_weekly_cov,weekly_beta
5796,2023-01-17,2023,3,MMM,124.926872,126.718767,119.943740,1.0,1.0,0.031148,0.013104,0.035274,0.0,0.035274,0.028391,0.000491,0.000596,1.214309
5797,2023-01-18,2023,3,MMM,121.127753,126.718767,119.943740,1.0,1.0,0.031148,0.013104,0.035274,0.0,0.035274,0.028391,0.000491,0.000596,1.214309
5798,2023-01-19,2023,3,MMM,116.864846,126.718767,119.943740,1.0,1.0,0.031148,0.013104,0.035274,0.0,0.035274,0.028391,0.000491,0.000596,1.214309
5799,2023-01-20,2023,3,MMM,119.055507,126.718767,119.943740,1.0,1.0,0.031148,0.013104,0.035274,0.0,0.035274,0.028391,0.000491,0.000596,1.214309
5805,2023-01-30,2023,5,MMM,111.092159,113.935974,118.897864,1.0,1.0,0.035380,-0.001059,-0.031438,0.0,-0.031438,0.022562,0.000373,0.000436,1.168082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584842,2023-06-12,2023,24,YUM,134.630000,132.211526,131.573586,1.0,2.0,0.014518,0.042583,0.049926,0.0,0.049926,0.014693,0.000094,0.000111,1.175260
2584843,2023-06-12,2023,24,ZBRA,279.990000,272.210026,294.525350,1.0,2.0,0.010932,0.024908,0.004967,0.0,0.004967,0.014693,0.000094,0.000167,1.768091
2584844,2023-06-12,2023,24,ZBH,135.550000,132.211526,129.178231,1.0,2.0,0.024419,0.024580,0.031954,0.0,0.031954,0.014693,0.000094,-0.000006,-0.058300
2584845,2023-06-12,2023,24,ZION,30.240000,30.965639,36.999472,1.0,2.0,0.037947,0.052400,-0.006572,0.0,-0.006572,0.014693,0.000094,0.000371,3.927887


In [26]:
parameter = {
    "value":True,
    "ceiling":True,
    "classification":False,
    "rank":False,
    "short":False,
    "market_return":1.5
}

In [27]:
stuff = test_portfolio.recommendation(sim.copy(),parameter)

# SET THE CURRENT WEEK

In [11]:
set_the_current_Week

NameError: name 'set_the_current_Week' is not defined

In [33]:
rec = stuff[0]
week = 24
trades = rec[rec["week"]==week].head(10).merge(sp500[["ticker","Security","GICS Sector"]],on="ticker")
trades[["year","week","ticker","Security","GICS Sector","weekly_delta","weekly_delta_sign"]]

,year,week,ticker,Security,GICS Sector,weekly_delta,weekly_delta_sign
0,2023,24,NVR,"NVR, Inc.",Consumer Discretionary,0.224841,1.0
1,2023,24,CTLT,Catalent,Health Care,0.125852,1.0
2,2023,24,AVGO,Broadcom Inc.,Information Technology,0.096541,1.0
3,2023,24,MPWR,Monolithic Power Systems,Information Technology,0.071368,1.0
4,2023,24,GWW,W. W. Grainger,Industrials,0.065353,1.0
5,2023,24,EPAM,EPAM Systems,Information Technology,0.063566,1.0
6,2023,24,ANET,Arista Networks,Information Technology,0.056237,1.0
7,2023,24,TER,Teradyne,Information Technology,0.051871,1.0
8,2023,24,MAS,Masco,Industrials,0.051687,1.0
9,2023,24,MCHP,Microchip Technology,Information Technology,0.051458,1.0


In [24]:
account = alp.paper_get_account()
cash = float(account.cash) - 10

In [26]:
alp.paper_trading_client.get_all_positions()

[Position(asset_id=UUID('74bfaf2c-9f8a-495d-b1ec-d701f492545a'), symbol='AMCR', exchange=<AssetExchange.NYSE: 'NYSE'>, asset_class=<AssetClass.US_EQUITY: 'us_equity'>, avg_entry_price='10.18', qty='982.220039292', side=<PositionSide.LONG: 'long'>, market_value='9959.71119842088', cost_basis='9998.999999993', unrealized_pl='-39.28880157212', unrealized_plpc='-0.0039292730845232', unrealized_intraday_pl='-39.28880157168', unrealized_intraday_plpc='-0.0039292730844794', current_price='10.14', lastday_price='10.03', change_today='0.0109670987038883'),
 Position(asset_id=UUID('7a8b4fcf-0622-47ef-8541-de7f70ffc0c4'), symbol='AVGO', exchange=<AssetExchange.NASDAQ: 'NASDAQ'>, asset_class=<AssetClass.US_EQUITY: 'us_equity'>, avg_entry_price='854.234271599', qty='11.705219905', side=<PositionSide.LONG: 'long'>, market_value='10018.61476888855', cost_basis='9998.999999448', unrealized_pl='19.61476944055', unrealized_plpc='0.0019616731114744', unrealized_intraday_pl='19.614769434759021905', unreal

In [17]:
fail

NameError: name 'fail' is not defined

In [22]:
order_data = []
for row in trades.iterrows():
    try:
        ticker = row[1]["ticker"]
        amount = round(cash / 10) 
        print(ticker,amount)
        order_data.append(alp.paper_market_order(ticker,amount))
        sleep(5)
    except Exception as e:
        print(str(e))
    

NVR 9999
CPB 9999
AMCR 9999
DG 9999
CHD 9999
AVGO 9999
CLX 9999
MCD 9999
BIO 9999
ILMN 9999


In [23]:
order_data

[Order(id=UUID('d5036e8a-8748-44a7-991a-d8f07a9c9446'), client_order_id='c2898381-4675-4afe-b41d-fc07a7344ee6', created_at=datetime.datetime(2023, 6, 13, 15, 1, 0, 35784, tzinfo=datetime.timezone.utc), updated_at=datetime.datetime(2023, 6, 13, 15, 1, 0, 35820, tzinfo=datetime.timezone.utc), submitted_at=datetime.datetime(2023, 6, 13, 15, 1, 0, 34830, tzinfo=datetime.timezone.utc), filled_at=None, expired_at=None, canceled_at=None, failed_at=None, replaced_at=None, replaced_by=None, replaces=None, asset_id=UUID('6521415f-8f5c-48be-ab46-c854cd21c7b7'), symbol='NVR', asset_class=<AssetClass.US_EQUITY: 'us_equity'>, notional='9999', qty=None, filled_qty='0', filled_avg_price=None, order_class=<OrderClass.SIMPLE: 'simple'>, order_type=<OrderType.MARKET: 'market'>, type=<OrderType.MARKET: 'market'>, side=<OrderSide.BUY: 'buy'>, time_in_force=<TimeInForce.DAY: 'day'>, limit_price=None, stop_price=None, status=<OrderStatus.PENDING_NEW: 'pending_new'>, extended_hours=False, legs=None, trail_per

In [ ]:
## close all
# close_data = alp.paper_close_all()